In [ ]:
import os
import pandas as pd
import numpy as np
import scipy

from datetime import timedelta as dt
pd.set_option("display.max_rows",None)
#os.chdir('C:\\Users\\somd7w\\Desktop\\DL_Projects\\preproc_cntr')
cwd = os.getcwd()
data =cwd+'/data'
output=cwd+'/output'

In [ ]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [ ]:
def read_sheet(a):
    try:
        df = pd.read_csv(csv_dict[str(a)], index_col=False)
    except:
        try:
            df = pd.read_csv(csv_dict[str(a)],index_col=False,encoding='latin-1')
        except:
            print('Error Opening CSV File')
    print("Columns of sheet: ",a)
    print(list(df))
    return df

def get_df_stats(df,drop_cols):
    # Drop rows if values are NaN in PAT_ID, CT_SCAN_DATE
    df = df.dropna(subset=drop_cols)
    # Unique PAT_IDs and Given_MRN:
    print("Number of rows in sheet = ",len(df))
    pat_ids = df.PAT_ID.unique()
    print("Number of unique PAT_IDs = ",len(pat_ids))
    mrns = df.GIVEN_MRN.unique()
    print("Number of unique MRNs = ",len(mrns))
   # accs = df.ACC.unique()
   # print("Number of unique ACC = ",len(accs))
    return(df)
    
def filter_duplicates(df,filter_col,sort_order):
    if sort_order == 'ascending':
     #   print('Sorting by ascending order and returning largest value')
        df = df.sort_values(by=[filter_col])
    else:
      #  print('Sorting by descending order and returning smallest value')
        df = df.sort_values(by=[filter_col],ascending=False)
    
    df = df.drop_duplicates('ACC', keep='last')
    return(df)

# Read CSV files from EPIC query, print and store as Dictionary

In [ ]:
csv_list = find_csv_filenames(data)
csv_dict = {}
print("List of Files from EPIC Query:")
for i,csv in enumerate(csv_list):
    print(i+1,': ',csv)
    csv_dict[str(i+1)] = os.path.join(data,csv)

In [ ]:
print(csv_dict)

In [ ]:
# Read age file
df_age = read_sheet('11')

In [ ]:
# Read Valid Patients Data 
validfile = output+'/corrected_whoosh_data_validpatients_v5.csv'
df_V = pd.read_csv(validfile, index_col=False)
df_V['CT_SCAN_DATE'] = pd.to_datetime(df_V.CT_SCAN_DATE)
print(pd.unique(df_V['PROBLEM_GROUP']))

In [ ]:
len(df_V)

In [ ]:
display(df_age.head(100))

In [ ]:
print(list(df_V))
display(df_V.head(100))

In [ ]:
# Find number of patients in both the sheets
print("Number of valid patients: ")
print(len(df_V))
print("Number of patients in Age sheet: ")
print(len(df_age))

df_age.rename(columns={'Accession Number': 'ACC'}, inplace=True)
df_age.rename(columns={'Age (At Exam)': 'Age'}, inplace=True)

print("Before dropping duplicates: ", len(df_age))
df_age = filter_duplicates(df_age,'ACC','ascending')

print("After dropping duplicates: ", len(df_age))
acc_age = [acc for acc in df_V.ACC.values if acc in df_age.ACC.values]
print("Number of valid patients with age info: ",len(acc_age))

In [ ]:
print(list(df_age))

In [ ]:
df_age = df_age[['ACC','Age','Sex','Race']]
df_V = pd.merge(df_V, df_age, on='ACC', how='inner')
print(len(df_V))
print(list(df_V))
df_V['AgeGroup'] = round(df_V[['Age']])
print(list(df_V))

In [ ]:
display(df_V.head(10))

In [ ]:
pd.to_numeric(df_V['Age'], downcast='integer',errors='ignore')
pd.options.display.float_format = '{:,.1f}'.format
#df_V['Age'] = df_V['Age'].astype('Int64')

# Sory by Age
#df_V = df_V.sort_values(by='Age',ascending=False)
#display(df_V['AgeGroup'].head(10))


print('Valid Exams', len(df_V))

pats_agelt20  = len(df_V[df_V['Age'] < 20])
pats_agegt20 = len(df_V[df_V['Age'] >= 20])
pats_noage = len(df_V[df_V['Age'].isin([np.nan])])

print('Valid Exams with age >= 20', pats_agegt20)
print('Valid Exams with age < 20', pats_agelt20)
print('Valid Exams with no age', pats_noage)

In [ ]:
print(pd.unique(df_V['AgeGroup']))
# drop patients above 19
df_V = df_V[df_V['Age'] < 20]
print(pd.unique(df_V['AgeGroup']))

In [ ]:
print(list(df_V))

In [ ]:
df_V_bmi = df_V.dropna(subset=['BMI_CALCULATED'])
print('Exams with BMI: ', len(df_V_bmi))
df_V_bmi2 = df_V.dropna(subset=['BMI_FOR_AGE_PERCENTILE'])
print('Exams with BMI: ', len(df_V_bmi2))
df_V_bmi3 = df_V.dropna(subset=['BMI_CALCULATED','BMI_FOR_AGE_PERCENTILE'])
print('Exams with BMI: ', len(df_V_bmi3))

In [ ]:
# Length of all exams
print('All valid exams: ', len(df_V))

# Calculate exams with BMI,
df_V_bmi = df_V.dropna(subset=['BMI_FOR_AGE_PERCENTILE'])
df_V_bmi.to_csv(output+'/corrected_finaldata_wAgeRace_wbmi.csv')
print('Exams with BMI: ', len(df_V_bmi))

# Calculate exams without BMI,
df_V_nobmi = df_V[df_V['BMI_FOR_AGE_PERCENTILE'].isna()]
df_V_nobmi.to_csv(output+'/corrected_finaldata_nobmi.csv')
print('Exams without BMI: ', len(df_V_nobmi))

# Calculate exams with Height and Weight but no BMI
df_V_H_W_nobmi = df_V_nobmi[(df_V_nobmi['HEIGHT'].notna()) & (df_V_nobmi['WEIGHT'].notna())]
print('Exams with H and W, no bmi:', len(df_V_H_W_nobmi))

# Calculate exams with Height alone but no BMI, Weight
df_V_H_noW_nobmi = df_V_nobmi[(df_V_nobmi['HEIGHT'].notna()) & (df_V_nobmi['WEIGHT'].isna())]
print('Exams with H, no W, no bmi:',len(df_V_H_noW_nobmi))

# Caculate exams with Weight alone but no BMI, Height
df_V_noH_W_nobmi = df_V_nobmi[(df_V_nobmi['HEIGHT'].isna()) & (df_V_nobmi['WEIGHT'].notna())]
print('Exams with  W, no H, no bmi:',len(df_V_noH_W_nobmi))

# Caculate exams with no Weight, no BMI, no Height
df_V_noH_noW_nobmi = df_V_nobmi[(df_V_nobmi['HEIGHT'].isna()) & (df_V_nobmi['WEIGHT'].isna())]
print('Exams with  no W, no H, no bmi:',len(df_V_noH_noW_nobmi))

In [ ]:
# deleting dataframes without weight and bmi:
del df_V_noH_noW_nobmi
del df_V_H_noW_nobmi

In [ ]:
# Calculate bmi for cases with heigh and weight, but no BMI
# Invoke Jimmy's bmi calculator routines
from height_age_weight_percentiles import *

wt_rec_len_table = build_cdc_stats_table(data+'/weight_for_recumbent_length.csv')
bmi_age_table = build_cdc_stats_table(data+'/bmi_for_age.csv')

def bmi_percentile_per_row(row):
    """This function is applied to each row, it calculates the Z-score for a given age in months, height, and weight"""
    
    # Loads data from spreadsheet, uses age in months, weight in kg, and length (height) in cm.
    weight_kg = row['WEIGHT'] #/ POUNDS_TO_KG
    length_cm = row['HEIGHT'] #* INCHES_TO_CM
    sex = 1 if row['Sex'] == 'Male' else 2
    age_months = age_in_years_to_months(row['Age'])
    
    # handles blanks
    if length_cm < 0 or weight_kg < 0:
        print("Neg Error: Height {} Weight {} Sex {}".format(length_cm,weight_kg,sex))
        return np.nan
    
    # if less than 2 years old, use weight for recumbent length instead of height
    # (infants can't stand up so they measure them laying down)
    if  age_months < 24:
        try:
            # the variables_for functions return the normal distribution (I think) variables for the 
            v = wt_rec_len_table.variables_for(sex, factor=length_cm)
        except KeyError:
            print("Error: ACC {} Age {} Height {} Weight {} Sex {}".format(row['ACC'],row['Age'],length_cm,weight_kg,sex))
            return np.nan
        
        return float(zscore_for_measurement(weight_kg, v.L, v.M, v.S))
    
    # otherwise, just calculate the standard bmi zscore
    else:
        height_m = length_cm / 100
        bmi = weight_kg / (height_m**2)
        
        v = bmi_age_table.variables_for(sex, factor=age_months)
        return (1-scipy.stats.norm.sf(abs(float(zscore_for_measurement(bmi, v.L, v.M, v.S)))))*100

In [ ]:
bmi_percentile = df_V_H_W_nobmi.apply(bmi_percentile_per_row, axis=1)
df_V_H_W_nobmi['BMI_FOR_AGE_PERCENTILE'] = bmi_percentile
print("Number of exams for which BMI percentile is missing, but Height and weight are present: ",len(df_V_H_W_nobmi))
df_V_H_W_nobmi = df_V_H_W_nobmi.dropna(subset=['BMI_FOR_AGE_PERCENTILE'])
print("Number of exams for which BMI percentile was calculated: ",len(df_V_H_W_nobmi))

In [ ]:
df_V_final = pd.concat([df_V_H_W_nobmi,df_V_bmi],axis=0)

In [ ]:
len(df_V_final)

In [ ]:
df_V.to_csv(output+'/corrected_finaldata_forquery.csv')
df_V_H_W_nobmi.to_csv(output+'/corrected_finaldata_withcalculatedbmi.csv')
df_V_final.to_csv(output+'/corrected_finaldata_withbmionly_v5.csv')

In [ ]:
# Visualize age histogram
import matplotlib.pyplot as plt
#d = np.random.laplace(loc=15, scale=3, size=500)

fig = plt.figure(figsize=(15,7), dpi=300)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
ax1 = plt.subplot2grid((1,2),(0,0))

agebins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
male = df_V_final[df_V_final['Sex']=='Male']['AgeGroup']
female = df_V_final[df_V_final['Sex']=='Female']['AgeGroup']

n, bins, patches = plt.hist(x=[male,female], bins = agebins, stacked=True,color=["blue", "red"],
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.ylabel('Number of Exams')
plt.xlabel('Age (Yrs)')
plt.title('Age wise distribution of normal patients with bmi')
plt.xlim(xmax=18,xmin=0)
plt.xticks(range(0,19))
plt.legend({'Male': "blue", 'Female': "red"})
#plt.text(23, 45, r'$\mu=15, b=3$')
maxfreq = max([n[0].max(),n[1].max()])
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
# Set x-axis limits.
ax1 = plt.subplot2grid((1,2),(0,1))
male = df_V[df_V['Sex']=='Male']['AgeGroup']
female = df_V[df_V['Sex']=='Female']['AgeGroup']
n, bins, patches = plt.hist(x=[male,female], bins = agebins, stacked=True,color=["blue", "red"],
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.ylabel('Number of Exams')
plt.xlabel('Age (Yrs)')
plt.title('Age wise distribution of normal patients with age and weight')
plt.xlim(xmax=18,xmin=0)
plt.xticks(range(0,19))
plt.legend({'Male': "blue", 'Female': "red"})

#plt.savefig(output+'hist.png')

In [ ]:
plt.savefig(output+'hist.png')

In [ ]:
print(output)